In [2]:
import numpy as np
import arepo
import sys
from tqdm import tqdm
import astropy.units as u
import h5py as h5
import glob
import os
from numba import njit
import re

import matplotlib.pyplot as plt

from sklearn.cluster import KMeans
import cProfile

In [2]:
sim_list = ['Nbody', 'phantom-vacuum-Sg20-Rc3.5']

basepath = '/n/holystore01/LABS/hernquist_lab/Users/abeane/starbar_runs/runs/'

def read_snap(idx, sim_idx, lvl='lvl3', parttype=[0], fields=['Coordinates', 'Masses', 'Velocities'],
              basepath = basepath):
    fname = basepath + sim_list[sim_idx] + '/' + lvl + '/output'
    return arepo.Snapshot(fname, idx, parttype=parttype, fields=fields, combineFiles=True)

In [6]:
@njit
def sort_by_id(chunk_ids, tot_ids, pos, vel, acc):
    # This goes through the chunk ids and matches up with total ids
    # but this assumes chunk ids and total ids are already sorted, which greatly
    # speeds things up.

    # also properly handles missing ids (e.g. in the case of stars)

    Nchunk = len(chunk_ids)
    pos_chunk = np.zeros((Nchunk, 3))
    vel_chunk = np.zeros((Nchunk, 3))
    acc_chunk = np.zeros((Nchunk, 3))
    
    itot = 0
    
    for ichunk in range(Nchunk):
        chk_id = chunk_ids[ichunk]
        
        while chk_id > tot_ids[itot]:
            itot += 1
        
        if chk_id == tot_ids[itot]:
            for j in range(3):
                pos_chunk[ichunk][j] = pos[itot][j]
                vel_chunk[ichunk][j] = vel[itot][j]
                acc_chunk[ichunk][j] = acc[itot][j]
        
        else:
            for j in range(3):
                pos_chunk[ichunk][j] = np.nan
                vel_chunk[ichunk][j] = np.nan
                acc_chunk[ichunk][j] = np.nan
        
    return pos_chunk, vel_chunk, acc_chunk

In [27]:
def _run_thread(path, name, idx_list, snap_id, id_chunks_disk, id_chunks_bulge, id_chunks_star=None, data_dir='data/'):
    
    h5out_list = []

    Nsnap = len(idx_list)
    
    # Create a temporary directory which will store each chunk of ids as a separate file.
    prefix = data_dir + name + '/tmp' + str(snap_id)
    if not os.path.isdir(prefix):
        os.mkdir(prefix)
    
    if id_chunks_star is None:
        has_stars = False
        id_chunks_star = [None for i in range(len(id_chunks_disk))]
    else:
        has_stars = True

    # Loop through each id chunk and create the file with temporary output.
    for i,(id_chunk_disk_list, id_chunk_bulge_list, id_chunk_star_list) in enumerate(tqdm(zip(id_chunks_disk, id_chunks_bulge, id_chunks_star))):
        Nids_disk = len(id_chunk_disk_list)
        Nids_bulge = len(id_chunk_bulge_list)
        if has_stars:
            Nids_star = len(id_chunk_star_list)

        fout = prefix + '/tmp' + str(i) + '.hdf5'
        h5out = h5.File(fout, mode='w')

        pos_disk = np.zeros((Nids_disk, Nsnap, 3))
        pos_bulge = np.zeros((Nids_bulge, Nsnap, 3))
        if has_stars:
            pos_star = np.zeros((Nids_star, Nsnap, 3))
        time = np.zeros(Nsnap)
        
        h5out.create_dataset("Time", data=time)

        h5out.create_dataset("PartType2/ParticleIDs", data=id_chunk_disk_list)
        h5out.create_dataset("PartType2/Coordinates", data=pos_disk)
        h5out.create_dataset("PartType2/Velocities", data=pos_disk)
        h5out.create_dataset("PartType2/Acceleration", data=pos_disk)

        h5out.create_dataset("PartType3/ParticleIDs", data=id_chunk_bulge_list)
        h5out.create_dataset("PartType3/Coordinates", data=pos_bulge)
        h5out.create_dataset("PartType3/Velocities", data=pos_bulge)
        h5out.create_dataset("PartType3/Acceleration", data=pos_bulge)

        if has_stars:
            h5out.create_dataset("PartType4/ParticleIDs", data=id_chunk_star_list)
            h5out.create_dataset("PartType4/Coordinates", data=pos_star)
            h5out.create_dataset("PartType4/Velocities", data=pos_star)
            h5out.create_dataset("PartType4/Acceleration", data=pos_star)

        h5out_list.append(h5out)
    
    # Now loop through each index, read the snapshot, then loop through each
    # id chunk and write to the relevant file.
    for i,idx in enumerate(tqdm(idx_list)):
        sn = read_snap(idx, 0, parttype=[2, 3, 4], 
                       fields=['Coordinates', 'Masses', 'Velocities', 'ParticleIDs', 'Acceleration'])
        # Sort phase space properties by ID
        key_disk = np.argsort(sn.part2.id)
        pos_disk = sn.part2.pos.value[key_disk]
        vel_disk = sn.part2.vel.value[key_disk]
        acc_disk = sn.part2.acce[key_disk]
        disk_ids_sorted = sn.part2.id[key_disk]

        key_bulge = np.argsort(sn.part3.id)
        pos_bulge = sn.part3.pos.value[key_bulge]
        vel_bulge = sn.part3.vel.value[key_bulge]
        acc_bulge = sn.part3.acce[key_bulge]
        bulge_ids_sorted = sn.part3.id[key_bulge]

        if has_stars and sn.NumPart_Total[4] > 0:
            key_star = np.argsort(sn.part4.id)
            pos_star = sn.part3.pos.value[key_star]
            vel_star = sn.part3.vel.value[key_star]
            acc_star = sn.part3.acce[key_star]

            star_ids_sorted = sn.part4.id[key_star]

        for j,(id_chunk_disk_list, id_chunk_bulge_list, id_chunk_star_list) in enumerate(zip(id_chunks_disk, id_chunks_bulge, id_chunks_star)):
            h5out_list[j]['Time'][i] = sn.Time.value
            
            pos_chunk_disk, vel_chunk_disk, acc_chunk_disk = sort_by_id(id_chunk_disk_list, disk_ids_sorted, pos_disk, vel_disk, acc_disk)
            h5out_list[j]['PartType2/Coordinates'][:,i,:] = pos_chunk_disk
            h5out_list[j]['PartType2/Velocities'][:,i,:] = vel_chunk_disk
            h5out_list[j]['PartType2/Acceleration'][:,i,:] = acc_chunk_disk
            
            pos_chunk_bulge, vel_chunk_bulge, acc_chunk_bulge = sort_by_id(id_chunk_bulge_list, bulge_ids_sorted, pos_bulge, vel_bulge, acc_bulge)
            h5out_list[j]['PartType3/Coordinates'][:,i,:] = pos_chunk_bulge
            h5out_list[j]['PartType3/Velocities'][:,i,:] = vel_chunk_bulge
            h5out_list[j]['PartType3/Acceleration'][:,i,:] = acc_chunk_bulge

            if has_stars:
                if sn.NumPart_Total[4] > 0:
                    pos_chunk_star, vel_chunk_star, acc_chunk_star = sort_by_id(id_chunk_star_list, star_ids_sorted, pos_star, vel_star, acc_star)
                else:
                    pos_chunk_star = vel_chunk_star = acc_chunk_star = np.full((len(id_chunk_star_list), 3), np.nan)
                h5out_list[j]['PartType4/Coordinates'][:,i,:] = pos_chunk_star
                h5out_list[j]['PartType4/Velocities'][:,i,:] = vel_chunk_star
                h5out_list[j]['PartType4/Acceleration'][:,i,:] = acc_chunk_star

    
    # Close h5 files.
    for i,_ in enumerate(id_chunks_disk):
        h5out_list[i].close()
    
    return None

In [28]:
def get_id_indices_chunks(nsnap, path, nchunk, nproc):
    indices = np.arange(nsnap)

#     sn = read_snap(path, indices[-1], parttype=[2, 3, 4])
    sn = read_snap(indices[-1], 0, parttype=[2, 3, 4], fields=['ParticleIDs'])
    ids_disk = sn.part2.id
    ids_disk = np.sort(ids_disk)

    ids_bulge = sn.part3.id
    ids_bulge = np.sort(ids_bulge)

    if sn.NumPart_Total[4] > 0:
        ids_star = sn.part4.id
        ids_star = np.sort(ids_star)
        id_chunks_star = np.array_split(ids_star, nchunk)
    else:
        id_chunks_star = None

    id_chunks_disk = np.array_split(ids_disk, nchunk)
    id_chunks_bulge = np.array_split(ids_bulge, nchunk)


    indices_chunks = np.array_split(indices, 4*nproc)

    return id_chunks_disk, id_chunks_bulge, id_chunks_star, indices_chunks

In [33]:
basepath = '/n/holystore01/LABS/hernquist_lab/Users/abeane/starbar_runs/runs/'
name = 'Nbody-lvl3'
path = basepath + 'Nbody/lvl3'
data_dir = 'data/'

# nsnap = len(glob.glob(path+'/output/snapdir*/*.0.hdf5'))
nsnap=200
nchunk = 64
nproc = 12

id_chunks_disk, id_chunks_bulge, id_chunks_star, indices_chunks = get_id_indices_chunks(nsnap, path, nchunk, nproc)

if not os.path.isdir(data_dir+name):
    os.mkdir(data_dir+name)

In [37]:
i = 24
cProfile.run("_run_thread(path, name, indices_chunks[i], i, id_chunks_disk, id_chunks_bulge, id_chunks_star)",
             filename='prof.dat')


0it [00:00, ?it/s]
3it [00:00, 22.45it/s]
7it [00:00, 25.10it/s]
10it [00:00, 23.86it/s]
12it [00:00, 20.95it/s]
15it [00:00, 20.72it/s]
19it [00:00, 23.55it/s]
23it [00:00, 26.23it/s]
27it [00:01, 27.77it/s]
31it [00:01, 29.66it/s]
35it [00:01, 31.24it/s]
39it [00:01, 32.46it/s]
43it [00:01, 28.98it/s]
47it [00:01, 27.49it/s]
51it [00:01, 29.15it/s]
55it [00:01, 27.83it/s]
58it [00:02, 26.47it/s]
62it [00:02, 28.62it/s]
64it [00:02, 26.91it/s]
100%|██████████| 4/4 [00:43<00:00, 10.90s/it]

In [47]:
sn = read_snap(0, 0, parttype=[2, 3])

In [49]:
sn.NumPart_Total[2] + sn.NumPart_Total[3]

7518464

In [64]:
t = h5.File('/n/home01/abeane/starbar/plots/bar_orbits/data/bar_orbit_phantom-vacuum-Sg20-Rc3.5-lvl4/bar_orbit_phantom-vacuum-Sg20-Rc3.5-lvl4.50.hdf5')

In [65]:
t['id_list'][16000]

47604429

In [66]:
t['bar_metrics']['47604429']

array([[1.01600000e+03, 8.02276871e-01, 8.27006903e-01, 2.82978373e+02,
        3.94348727e-01, 6.60645161e+01],
       [1.01900000e+03, 8.13126471e-01, 8.79868693e-01, 2.82942535e+02,
        3.95466658e-01, 6.60645161e+01],
       [1.02300000e+03, 8.77403635e-01, 7.54585575e-01, 2.82939149e+02,
        3.74116267e-01, 6.60645161e+01],
       ...,
       [1.59300000e+03, 9.56763305e-01, 2.13942696e+01, 2.93763134e+02,
        9.88629475e-02, 1.02400000e+02],
       [1.59500000e+03, 9.56763305e-01, 2.13942696e+01, 2.93763134e+02,
        9.88629475e-02, 1.02400000e+02],
       [1.59900000e+03, 9.56763305e-01, 2.13942696e+01, 2.93763134e+02,
        9.88629475e-02, 4.99512195e+01]])

In [16]:
t = h5.File('/n/home01/abeane/starbar/plots/phase_space/data/phantom-vacuum-Sg20-Rc3.5-lvl3-old/phase_space_phantom-vacuum-Sg20-Rc3.5-lvl3.3.hdf5', 
            mode='r')

In [23]:
print(t['PartType4']['Velocities'][:6,:,:])

[[[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]
  ...
  [0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]]


In [3]:
base = '/n/home01/abeane/starbar/plots/phase_space/data/phantom-vacuum-Sg20-Rc3.5-lvl3-old/'
base = base + 'phase_space_phantom-vacuum-Sg20-Rc3.5-lvl3.'
for i in tqdm(range(48, 256)):
    fname = base + str(i) + '.hdf5'
    t = h5.File(fname, mode='r+')
    for key in ['Coordinates', 'Velocities', 'Acceleration']:
        t['PartType4'][key][:6,:,:] = np.nan
    t.close()

100%|██████████| 208/208 [09:38<00:00,  3.08s/it]
